# Orbit for Bayesian Time Series Forecasting in Python

In [ ]:
!pip install orbit-ml

  Using cached orbit-ml-1.1.4.7.tar.gz (447 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached cmdstanpy-1.2.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached pyro_ppl-1.9.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached stanio-0.5.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached pyro_api-0.1.2-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached cmdstanpy-1.2.5-py3-none-any.whl (94 kB)
Using cached pyro_ppl-1.9.1-py3-none-any.whl (755 kB)
Using cached pyro_api-0.1.2-py3-none-any.whl (11 kB)
Using cached stanio-0.5.1-py3-none-any.whl (8.1 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)


In [4]:
import pandas as pd
from orbit.utils.dataset import load_iclaims

# Load sample data
data = load_iclaims()
data["week"] = pd.to_datetime(data["week"])
data = data.rename(columns={"week": "date", "claims": "value"})

print(data.head())

ModuleNotFoundError: No module named 'orbit'

In [ ]:
from orbit.models import DLT
from orbit.diagnostics.plot import plot_predicted_data

# Initialize the DLT model
model = DLT(
    response_col="value",
    date_col="date",
    seasonality=52,  # Weekly seasonality
)

# Fit the model
model.fit(train_df=data)

In [ ]:
# Predict future values
predictions = model.predict(df=data)

# Plot the predicted data
plot_predicted_data(data, predictions, date_col="date", actual_col="value", pred_col="prediction")



In [ ]:
# Extract prediction intervals
predictions[["prediction", "prediction_5", "prediction_95"]].head()

In [ ]:
# Initialize a model with custom parameters
model = DLT(
    response_col="value",
    date_col="date",
    seasonality=52,
    damped=True,  # Damped trend
)
model.fit(train_df=data)

In [ ]:
from orbit.models import KTR

# Use Kernel Trend Regression (KTR) with custom priors
model = KTR(
    response_col="value",
    date_col="date",
    seasonality=52,
    level_knot_prior=0.5,
)
model.fit(train_df=data)

In [ ]:
from orbit.diagnostics.metrics import smape, rmse

# Calculate SMAPE and RMSE
true_values = data["value"]
predicted_values = predictions["prediction"]
print("SMAPE:", smape(true_values, predicted_values))
print("RMSE:", rmse(true_values, predicted_values))

In [ ]:
# Add external regressors (e.g., economic indicators)
data["recession"] = [1 if x % 12 < 3 else 0 for x in range(len(data))]  # Simulated example

# Initialize the model with regressors
model = DLT(
    response_col="value",
    date_col="date",
    seasonality=52,
    regressor_col=["recession"]
)
model.fit(train_df=data)